In [ ]:
!pip install --upgrade google-search-results
!pip install --upgrade openai
!pip install --upgrade azure-core
!pip install --upgrade requests
!pip install --upgrade beautifulsoup4

In [ ]:
import os, time, datetime, requests, bs4, re, urllib.parse
from typing import List
from openai import AzureOpenAI
from serpapi import GoogleSearch
from openai._exceptions import RateLimitError

os.environ["SERPAPI_KEY"]  = "" #Paste your SerpAPI key here.
os.environ["AZURE_OPENAI_KEY"] = "" #Paste your Azure AI Foundry API Key here.

# ── API KEYS ─────────────────────────────────────
SERP_API_KEY = os.getenv("SERPAPI_KEY")
AZURE_KEY    = os.getenv("AZURE_OPENAI_KEY")

client = AzureOpenAI(
    api_key=AZURE_KEY,
    api_version="", #Copy and paste the version listed on deployments on your Azure AI Foundry Deployments page.
    azure_endpoint="", #Copy and paste your endpoint that ends with .azure.com
)

# ── LLM helper ───────────────────────────────────
def chat(msgs, T=0.7, retries=3):
    for _ in range(retries):
        try:
            r = client.chat.completions.create(
                model="itai-2376", messages=msgs,
                max_tokens=512, temperature=T, top_p=1.0
            )
            return r.choices[0].message.content
        except RateLimitError as e:
            time.sleep(getattr(e, "retry_after", 5))
    raise RuntimeError("LLM rate‑limit")

# ── SEARCH LAYERS ───────────────────────────────
def serp_search(q:str, k:int=3)->List[dict]:
    if not SERP_API_KEY: return []
    try:
        d = GoogleSearch({"q":q,"api_key":SERP_API_KEY,"num":k,"hl":"en"}).get_dict()
        return [{"title":r["title"],"url":r["link"]}
                for r in d.get("organic_results",[])[:k]]
    except Exception:
        return []

DDG = "https://lite.duckduckgo.com/50x/?q={q}"
UA  = ("Mozilla/5.0 (iPhone; CPU iPhone OS 15_0) AppleWebKit/605.1.15 "
       "(KHTML, like Gecko) Version/15.0 Mobile/15E148 Safari/604.1")
def ddg_search(q:str, k:int=3)->List[dict]:
    html=requests.get(DDG.format(q=urllib.parse.quote_plus(q)),
                      headers={"User-Agent":UA},timeout=10).text
    soup=bs4.BeautifulSoup(html,"html.parser")
    links=soup.select("a.result-link--title") or soup.select(".result-link, .result-link--news")
    return [{"title":re.sub(r"\s+"," ",a.get_text(' ',strip=True)),"url":a['href']}
            for a in links[:k]]

def web_search(q:str, k:int=3)->str:
    today=datetime.date.today().isoformat()
    hit=serp_search(q,k)
    if hit:
        print("🔍 SerpAPI hit")
        return "\n".join(f"[{i}] {h['title']} – {h['url']} ({today})"
                         for i,h in enumerate(hit,1))
    hit=ddg_search(q,k)
    if hit:
        print("🌐 DuckDuckGo hit")
        return "\n".join(f"[{i}] {h['title']} – {h['url']} ({today})"
                         for i,h in enumerate(hit,1))
    return f"[1] No results found ({today})"

# ── ReAct agent (SEARCH → SUMMARIZE) ────────────
def research(question:str, max_turns:int=6):
    msgs=[{"role":"system","content":
           ("First output SEARCH:<keywords>. After I return the sources, output "
            "SUMMARIZE:<draft answer>.\n"
            "Use inline citations [1] [2] … and copy the numbered list beneath "
            "a 'Sources:' heading.")},
          {"role":"user","content":question}]
    stage, turns, last_text, sources_block = 0, 0, "", ""

    while True:
        if turns >= max_turns:
            return last_text or "⚠️ No summarized answer after retries."
        resp=chat(msgs); turns+=1; last_text=resp
        first=resp.lstrip().splitlines()[0].strip()

        # ---- SEARCH ----
        if stage==0 and first.startswith("SEARCH:"):
            sources_block = web_search(first[7:].strip())
            msgs[-1:]=[
                {"role":"assistant","content":sources_block},
                {"role":"assistant",
                 "content":("Now output SUMMARIZE:<draft answer> with inline citations "
                            "and include the numbered list under 'Sources:'.")}
            ]
            stage=1; continue

        # ---- SUMMARIZE ----
        if stage==1 and first.startswith("SUMMARIZE:"):
            draft = first[10:].strip()
            if "Sources:" not in draft:               # auto‑append if missing
                draft += "\n\nSources:\n" + sources_block
            return draft

        # ---- remind when off‑track ----
        reminder = "Begin with SEARCH:<keywords>." if stage==0 else "Awaiting SUMMARIZE:<draft>."
        msgs.append({"role":"assistant","content":reminder})

## Ask your questions here.
# ── Demo -----------------------------------------
if __name__=="__main__":
    answer = research("Who is George Washgington?")
    print("\n✅ FINAL ANSWER:\n", answer)


🔍 SerpAPI hit

✅ FINAL ANSWER:
 George Washington, the first President of the United States, served two terms from 1789 to 1797. Born on February 22, 1732, in Westmoreland County, Virginia, he was a pivotal figure in the founding of the nation. Prior to his presidency, Washington played a crucial role as the Commander-in-Chief of the Continental Army during the American Revolutionary War, leading the colonies to victory against the British [1][2]. His presidency set many precedents for the national government and the future officeholders. Washington's leadership style and decisions during his presidency established many forms of protocol and decorum that are still followed today [1][2][3]. Despite facing various challenges, such as the Whiskey Rebellion, his administration succeeded in laying down the strong foundations of the new government in line with the US Constitution. Washington retired after his second term, establishing the tradition of a peaceful transfer of power [2][3].

So